In [ ]:
!pip install selenium

In [107]:
import selenium, pandas as pd
import time
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

In [108]:
print(dir(Keys))

['ADD', 'ALT', 'ARROW_DOWN', 'ARROW_LEFT', 'ARROW_RIGHT', 'ARROW_UP', 'BACKSPACE', 'BACK_SPACE', 'CANCEL', 'CLEAR', 'COMMAND', 'CONTROL', 'DECIMAL', 'DELETE', 'DIVIDE', 'DOWN', 'END', 'ENTER', 'EQUALS', 'ESCAPE', 'F1', 'F10', 'F11', 'F12', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'HELP', 'HOME', 'INSERT', 'LEFT', 'LEFT_ALT', 'LEFT_CONTROL', 'LEFT_SHIFT', 'META', 'MULTIPLY', 'NULL', 'NUMPAD0', 'NUMPAD1', 'NUMPAD2', 'NUMPAD3', 'NUMPAD4', 'NUMPAD5', 'NUMPAD6', 'NUMPAD7', 'NUMPAD8', 'NUMPAD9', 'PAGE_DOWN', 'PAGE_UP', 'PAUSE', 'RETURN', 'RIGHT', 'SEMICOLON', 'SEPARATOR', 'SHIFT', 'SPACE', 'SUBTRACT', 'TAB', 'UP', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']


In [ ]:
ActionChains.key_up?

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from selenium import webdriver

In [130]:
#Chrome web driver location provided for selenium automation works 
driver=webdriver.Chrome(r"C:\Users\smahi\Downloads\chromedriver_win32\chromedriver.exe")
#opening the olx web link through the chrome driver
driver.get('https://www.olx.in/')
#finding the xpath for cars
car=driver.find_element_by_xpath("//div[@class='_2xg5B']/div[@class='_3AGJR _18NX_']/a[@class='_2fitb']")
#clicking on the 'cars'
car.click()
time.sleep(5)

In [ ]:
# getting the state names from the olx website
location=driver.find_elements_by_xpath('//a[@class="_396bV _2zs4P _1nXYG"]')
prices=[]
date=[]
area=[]
company=[]
name=[]
year=[]
km=[]
state=[]
for i in range(1,len(location)):
    if i<=5:
        k=driver.find_element_by_xpath('//ul[@class="_3eZBP"]/li['+str(i)+']/a[@class="_396bV _2zs4P _1nXYG"]')
    else:
        j=i-5
        print(j)
        k=driver.find_element_by_xpath('//div[1]/li['+str(j)+']/a[@class="_396bV _2zs4P _1nXYG"]')
    #state name
    St=location[i-1].text.split()[0]
    #no of cars in that state were posted for add 
    nc=location[i-1].text.split()[1].replace('(','').replace(')','')
    Nc=''
    for j in nc.split(','):
        Nc=Nc+j
    Nc
    #clicking on that specfic state for going to collect the car details
    k.click()
    time.sleep(5)
    if int(Nc)<=1000:
        M=int(Nc)
    else:
        M=1000
    while len(prices)<10:
        # finding the car price tag
        price=driver.find_elements_by_xpath('//div[@class="_31uw8"]/span[@class="_3GOwr"]')
        for i in price:
            prices.append(i.text.split(' ')[1])
        # collecting the company name and car name tag
        cn=driver.find_elements_by_xpath('//div[@class="_31uw8"]/div[@class="_4aNdc"]')
        for i in cn:
            if i.text.split(' ')[0]=='Maruti':
                company.append('Maruti Suzuki')
                I=''
                for i in i.text.split(' ')[2:]:
                    I=I+i+' '
                    I.strip()
                name.append(I.strip())
            else:
                company.append(i.text.split(' ')[0])
                I=''
                for i in i.text.split(' ')[1:]:
                    I=I+i+' '
                name.append(I.strip())
        #finding the year and kilometers driven tag
        yk=driver.find_elements_by_xpath('//div[@class="_31uw8"]/div[@class="KFHpP"]')
        for i in yk:
            year.append(i.text.split()[0])
            km.append(i.text.split()[-2])
        #finding the area and date of add posted tags
        ad=driver.find_elements_by_xpath('//div[@class="_31uw8"]/div[@class="_1zvfB"]')
        for i in ad:
            area.append(i.text.split('\n')[0])
            state.append(St)
            try:
                date.append(i.text.split('\n')[1])
            except IndexError:
                date.append('no date') 
        # finding the tag for next page button
        np=driver.find_element_by_xpath('//button[@class="rui-39-wj rui-3evoE rui-1JPTg"]')
        #clicking on the next page button
        np.click()
        time.sleep(5)
        print(len(prices))
driver.find_element_by_xpath('//div[@class="_3zgcG"]').click()
        

In [155]:
# getting the state names from the olx website
location=driver.find_elements_by_xpath('//a[@class="_396bV _2zs4P _1nXYG"]')
prices=[]
date=[]
area=[]
company=[]
name=[]
year=[]
km=[]
state=[]
o=10
for i in range(1,len(location)):
    if i<=5:
        k=driver.find_element_by_xpath('//ul[@class="_3eZBP"]/li['+str(i)+']/a[@class="_396bV _2zs4P _1nXYG"]')
    else:
        j=i-5
        print(j)
        k=driver.find_element_by_xpath('//div[1]/li['+str(j)+']/a[@class="_396bV _2zs4P _1nXYG"]')
    #state name
    #no of cars in that state were posted for add 
    Nc=''
    for j in nc.split(','):
        Nc=Nc+j
    Nc
    #clicking on that specfic state for going to collect the car details
    k.click()
    time.sleep(5)
    if int(Nc)<=1000:
        M=int(Nc)
    else:
        M=1000
    o=o+len(prices)
    while len(prices)<o:
        # finding the car price tag
        price=driver.find_elements_by_xpath('//div[@class="_31uw8"]/span[@class="_3GOwr"]')
        for i in price:
            prices.append(i.text.split(' ')[1])
        # collecting the company name and car name tag
        cn=driver.find_elements_by_xpath('//div[@class="_31uw8"]/div[@class="_4aNdc"]')
        for i in cn:
            if i.text.split(' ')[0]=='Maruti':
                company.append('Maruti Suzuki')
                I=''
                for i in i.text.split(' ')[2:]:
                    I=I+i+' '
                name.append(I.strip())
            else:
                company.append(i.text.split(' ')[0])
                I=''
                for i in i.text.split(' ')[1:]:
                    I=I+i+' '
                name.append(I.strip())
        #finding the year and kilometers driven tag
        yk=driver.find_elements_by_xpath('//div[@class="_31uw8"]/div[@class="KFHpP"]')
        for i in yk:
            year.append(i.text.split()[0])
            km.append(i.text.split()[-2])
        #finding the area and date of add posted tags
        ad=driver.find_elements_by_xpath('//div[@class="_31uw8"]/div[@class="_1zvfB"]')
        for i in ad:
            area.append(i.text.split('\n')[0])
            try:
                date.append(i.text.split('\n')[1])
            except IndexError:
                date.append('no date') 
        # finding the tag for next page button
        np=driver.find_element_by_xpath('//button[@class="rui-39-wj rui-3evoE rui-1JPTg"]')
        #clicking on the next page button
        np.click()
        print(len(prices))
        time.sleep(5)
    driver.find_element_by_xpath('//div[@class="rui-3unfU"]').click()
    driver.find_element_by_xpath('//div[@class="_3zgcG"]').click()
    

40
80
120
200
240
320
440
480
560
680
840
1
880
960
1080
1240
1440
1679
2
1719
1799
1919
2079
2278
2517
2796
3115
3474
3
3514
3594
3713
3872
4071
4310
4588
4905
5262
5659
6096


WebDriverException: Message: disconnected: Unable to receive message from renderer
  (Session info: chrome=97.0.4692.71)
Stacktrace:
Backtrace:
	Ordinal0 [0x00BEFDC3+2555331]
	Ordinal0 [0x00B877F1+2127857]
	Ordinal0 [0x00A82E08+1060360]
	Ordinal0 [0x00A75461+1004641]
	Ordinal0 [0x00A743FA+1000442]
	Ordinal0 [0x00A747C7+1001415]
	Ordinal0 [0x00A8ABC0+1092544]
	Ordinal0 [0x00A8505B+1069147]
	Ordinal0 [0x00A84BD5+1067989]
	Ordinal0 [0x00A853AB+1069995]
	Ordinal0 [0x00A85650+1070672]
	Ordinal0 [0x00AAE212+1237522]
	Ordinal0 [0x00AAE69B+1238683]
	Ordinal0 [0x00AD928C+1413772]
	Ordinal0 [0x00AC7B54+1342292]
	Ordinal0 [0x00AD75FA+1406458]
	Ordinal0 [0x00AC7976+1341814]
	Ordinal0 [0x00AA36B6+1193654]
	Ordinal0 [0x00AA4546+1197382]
	GetHandleVerifier [0x00D89622+1619522]
	GetHandleVerifier [0x00E3882C+2336844]
	GetHandleVerifier [0x00C823E1+541697]
	GetHandleVerifier [0x00C81443+537699]
	Ordinal0 [0x00B8D18E+2150798]
	Ordinal0 [0x00B91518+2168088]
	Ordinal0 [0x00B91660+2168416]
	Ordinal0 [0x00B

In [151]:
len(prices)

40

In [153]:
driver.find_element_by_xpath('//div[@class="rui-3unfU"]').click()

In [ ]:
location=driver.find_elements_by_xpath('//a[@class="_396bV _2zs4P _1nXYG"]')

In [ ]:
location[2].text

In [ ]:
driver.find_element_by_xpath('//ul[@class="_3eZBP"]/li[4]/a[@class="_396bV _2zs4P _1nXYG"]').click()

In [ ]:
driver.find_element_by_xpath('//div[@class="_3zgcG"]').click()

In [ ]:
driver.find_element_by_xpath('//div[@class="_3zgcG"]').click()
for i in range(1,len(location)):
    if i<=5:
        k=driver.find_element_by_xpath('//ul[@class="_3eZBP"]/li['+str(i)+']/a[@class="_396bV _2zs4P _1nXYG"]')
        k.text
    else:
        j=i-5
        print(j)
        k=driver.find_element_by_xpath('//div[1]/li['+str(j)+']/a[@class="_396bV _2zs4P _1nXYG"]')
        k.text
    k.click()
    time.sleep(1)
    
    driver.find_element_by_xpath('//div[@class="_3zgcG"]').click()
    

In [ ]:
for i in range(1,len(location)):
    if i<=5:
        k=driver.find_element_by_xpath('//ul[@class="_3eZBP"]/li['+str(i)+']/a[@class="_396bV _2zs4P _1nXYG"]')
    else:
        j=i-5
        k=driver.find_element_by_xpath('//div[1]/li['+str(j)+']/a[@class="_396bV _2zs4P _1nXYG"]').click()

In [ ]:
len(location)

In [ ]:
driver.find_element_by_xpath('//div[1]/li[7]/a[@class="_396bV _2zs4P _1nXYG"]').click()

In [ ]:
prices[0]

In [ ]:
cn=driver.find_elements_by_xpath('//div[@class="_31uw8"]/div[@class="_4aNdc"]')

In [ ]:
company=[]
name=[]
for i in cn:
    if i.text.split(' ')[0]=='Maruti':
        company.append('Maruti Suzuki')
        I=''
        for i in i.text.split(' ')[2:]:
            I=I+i+' '
            I.strip()
        name.append(I.strip())
    else:
        company.append(i.text.split(' ')[0])
        I=''
        for i in i.text.split(' ')[1:]:
            I=I+i+' '
        name.append(I.strip())
        

In [ ]:
yk=driver.find_elements_by_xpath('//div[@class="_31uw8"]/div[@class="KFHpP"]')

In [ ]:
year=[]
km=[]
for i in yk:
    year.append(i.text.split()[0])
    km.append(i.text.split()[-2])

In [ ]:
ad=driver.find_elements_by_xpath('//div[@class="_31uw8"]/div[@class="_1zvfB"]')
area=[]
date=[]
state=[]
for i in ad:
    area.append(i.text.split('\n')[0])
    date.append(i.text.split('\n')[1])
    state.append(St)
    

In [ ]:
np=driver.find_element_by_xpath('//button[@class="rui-39-wj rui-3evoE rui-1JPTg"]')

In [123]:
driver.find_element_by_xpath('//div[@class="rui-3unfU"]').click()

In [104]:
ActionChains.key_up?

In [ ]:
i.text

In [120]:
ActionChains(driver, duration=10000).key_up(Keys.ARROW_UP).perform()